In [2]:
from urllib.request import urlopen
dat = urlopen("https://www.investopedia.com/articles/investing/053116/10-largest-holdings-sp-500-aaplamznfb.asp").read()
import re
matches  = re.findall("\(([A-Z]+)\)",str(dat))
st = matches[5:] 
st[0] ='MSFT'
tickers = st

In [3]:
import datetime
import pandas as pd
import numpy as np
from pandas_datareader import data as wb

start = '2018-01-01'
end = '2019-01-01'

thelen = len(tickers)

data = pd.DataFrame(columns = tickers)
for ticker in tickers:
    data[ticker] = wb.DataReader(ticker, start = start, end = end, data_source='yahoo')['Adj Close']
data.head()

,MSFT,AAPL,AMZN,GOOG,GOOGL,FB,JNJ,V,PG,JPM,UNH,MA,INTC,VZ,HD,T,PFE,MRK,PEP,RSP
Date,,,,,,,,,,,,,,,,,,,,
2018-01-02,82.798996,166.054321,1189.010010,1065.000000,1073.209961,181.419998,130.099716,112.504623,83.576973,99.667267,212.746506,149.622391,43.822281,47.474491,176.754425,32.630741,32.927341,52.251068,109.380081,96.773682
2018-01-03,83.184326,166.025375,1204.199951,1082.479980,1091.520020,184.669998,131.342499,113.624641,83.475563,99.768829,214.978226,151.503586,42.335033,46.498932,177.675644,31.877203,33.171303,52.176716,109.092873,97.268097
2018-01-04,83.916473,166.796555,1209.589966,1086.400024,1095.760010,184.330002,131.333145,114.047112,84.065620,101.198090,215.911316,153.463608,41.558678,46.649700,179.085724,32.173538,33.243599,53.022469,109.630226,97.600891
2018-01-05,84.956886,168.695572,1229.140015,1102.229980,1110.290039,186.850006,132.417084,116.778435,84.120949,100.548431,220.028488,156.644974,41.848644,46.543266,180.956360,32.292076,33.306850,52.966705,109.945236,98.066788
2018-01-08,85.043571,168.069016,1246.869995,1106.939941,1114.209961,188.279999,132.585281,117.250008,84.563507,100.696907,216.209503,157.118515,41.848644,46.463455,180.523941,32.385208,32.936375,52.660000,109.315224,98.390068


In [4]:
returns = data.apply(lambda x: np.log(x)-np.log(x.shift(1)))
returns.dropna(axis=0, inplace = True)
returns.head()

,MSFT,AAPL,AMZN,GOOG,GOOGL,FB,JNJ,V,PG,JPM,UNH,MA,INTC,VZ,HD,T,PFE,MRK,PEP,RSP
Date,,,,,,,,,,,,,,,,,,,,
2018-01-03,0.004643,-0.000174,0.012694,0.016280,0.016917,0.017756,0.009507,0.009906,-0.001214,0.001018,0.010435,0.012495,-0.034527,-0.020763,0.005198,-0.023364,0.007382,-0.001424,-0.002629,0.005096
2018-01-04,0.008763,0.004634,0.004466,0.003615,0.003877,-0.001843,-0.000071,0.003711,0.007044,0.014224,0.004331,0.012854,-0.018509,0.003237,0.007905,0.009253,0.002177,0.016079,0.004914,0.003416
2018-01-05,0.012322,0.011321,0.016033,0.014466,0.013173,0.013579,0.008219,0.023667,0.000658,-0.006440,0.018889,0.020518,0.006953,-0.002284,0.010391,0.003678,0.001901,-0.001052,0.002869,0.004762
2018-01-08,0.001020,-0.003721,0.014322,0.004264,0.003524,0.007624,0.001269,0.004030,0.005247,0.001476,-0.017509,0.003018,0.000000,-0.001716,-0.002392,0.002880,-0.011185,-0.005807,-0.005747,0.003291
2018-01-09,-0.000680,-0.000115,0.004665,-0.000614,-0.001275,-0.002180,0.015733,-0.001929,-0.007332,0.005056,0.004971,0.001443,-0.025352,-0.003675,0.005504,-0.006911,-0.001098,0.002468,0.000763,0.000483


In [5]:
class CLA:  
    def __init__(self,mean,covar,lB,uB):  
        # Initialize the class  
        self.mean=mean  
        self.covar=covar  
        self.lB=lB  
        self.uB=uB  
        self.w=[] # solution  
        self.l=[] # lambdas  
        self.g=[] # gammas  
        self.f=[] # free weights  
#---------------------------------------------------------------  
    def solve(self):  
        # Compute the turning points,free sets and weights  
        f,w=self.initAlgo()  
        self.w.append(np.copy(w)) # store solution  
        self.l.append(None)  
        self.g.append(None)  
        self.f.append(f[:])  
        while True:  
            #1) case a): Bound one free weight  
            l_in=None  
            if len(f)>1:  
                covarF,covarFB,meanF,wB=self.getMatrices(f)  
                covarF_inv=np.linalg.inv(covarF)  
                j=0  
                for i in f:  
                    l,bi=self.computeLambda(covarF_inv,covarFB,meanF,wB,j,[self.lB[i],self.uB[i]])  
                    if (l_in is None or l>l_in):l_in,i_in,bi_in=l,i,bi  
                    j+=1  
            #2) case b): Free one bounded weight  
            l_out=None  
            if len(f)<self.mean.shape[0]:  
                b=self.getB(f)  
                for i in b:  
                    covarF,covarFB,meanF,wB=self.getMatrices(f+[i])  
                    covarF_inv=np.linalg.inv(covarF)  
                    l,bi=self.computeLambda(covarF_inv,covarFB,meanF,wB,meanF.shape[0]-1, 
                        self.w[-1][i]) ##########################  
                    if (self.l[-1]==None or l<self.l[-1]) and (l_out is None or l>l_out):l_out,i_out=l,i  
            if (l_in==None or l_in<0) and (l_out==None or l_out<0):  
                #3) compute minimum variance solution  
                self.l.append(0)  
                covarF,covarFB,meanF,wB=self.getMatrices(f)  
                covarF_inv=np.linalg.inv(covarF)  
                meanF=np.zeros(meanF.shape)  
            else:  
                #4) decide lambda  
                if l_out is None or (l_in is not None and l_in>l_out):  
                    self.l.append(l_in)  
                    f.remove(i_in)  
                    w[i_in]=bi_in # set value at the correct boundary  
                else:  
                    self.l.append(l_out)  
                    f.append(i_out)  
                covarF,covarFB,meanF,wB=self.getMatrices(f)  
                covarF_inv=np.linalg.inv(covarF)  
            #5) compute solution vector  
            wF,g=self.computeW(covarF_inv,covarFB,meanF,wB)  
            for i in range(len(f)):w[f[i]]=wF[i]  
            self.w.append(np.copy(w)) # store solution  
            self.g.append(g)  
            self.f.append(f[:])  
            if self.l[-1]==0:break  
        #6) Purge turning points  
        self.purgeNumErr(10e-10)  
        self.purgeExcess()  
#---------------------------------------------------------------  
    def initAlgo(self):  
        # Initialize the algo  
        #1) Form structured array  
        a=np.zeros((self.mean.shape[0]),dtype=[('id',int),('mu',float)])  
        b=[self.mean[i][0] for i in range(self.mean.shape[0])] # dump array into list  
        a[:]=list(zip(range(self.mean.shape[0]),b)) # fill structured array  
        #2) Sort structured array  
        b=np.sort(a,order='mu')  
        #3) First free weight  
        i,w=b.shape[0],np.copy(self.lB)  
        while sum(w)<1:  
            i-=1  
            w[b[i][0]]=self.uB[b[i][0]]  
        w[b[i][0]]+=1-sum(w)  
        return [b[i][0]],w  
#---------------------------------------------------------------  
    def computeBi(self,c,bi):  
        if c>0:  
            bi=bi[1][0]  
        if c<0:  
            bi=bi[0][0]  
        return bi  
#---------------------------------------------------------------  
    def computeW(self,covarF_inv,covarFB,meanF,wB):  
        #1) compute gamma  
        onesF=np.ones(meanF.shape)  
        g1=np.dot(np.dot(onesF.T,covarF_inv),meanF)  
        g2=np.dot(np.dot(onesF.T,covarF_inv),onesF)  
        if wB is None:  
            g,w1=float(-self.l[-1]*g1/g2+1/g2),0  
        else:  
            onesB=np.ones(wB.shape)  
            g3=np.dot(onesB.T,wB)  
            g4=np.dot(covarF_inv,covarFB)  
            w1=np.dot(g4,wB)  
            g4=np.dot(onesF.T,w1)  
            g=float(-self.l[-1]*g1/g2+(1-g3+g4)/g2)  
        #2) compute weights  
        w2=np.dot(covarF_inv,onesF)  
        w3=np.dot(covarF_inv,meanF)  
        return -w1+g*w2+self.l[-1]*w3,g  
#---------------------------------------------------------------  
    def computeLambda(self,covarF_inv,covarFB,meanF,wB,i,bi):  
        #1) C  
        onesF=np.ones(meanF.shape)  
        c1=np.dot(np.dot(onesF.T,covarF_inv),onesF)  
        c2=np.dot(covarF_inv,meanF)  
        c3=np.dot(np.dot(onesF.T,covarF_inv),meanF)  
        c4=np.dot(covarF_inv,onesF)  
        c=-c1*c2[i]+c3*c4[i]  
        if c==0:return  
        #2) bi  
        if type(bi)==list:bi=self.computeBi(c,bi)  
        #3) Lambda  
        if wB is None:  
            # All free assets  
            return float((c4[i]-c1*bi)/c),bi  
        else:  
            onesB=np.ones(wB.shape)  
            l1=np.dot(onesB.T,wB)  
            l2=np.dot(covarF_inv,covarFB)  
            l3=np.dot(l2,wB)  
            l2=np.dot(onesF.T,l3)  
            return float(((1-l1+l2)*c4[i]-c1*(bi+l3[i]))/c),bi  
#---------------------------------------------------------------  
    def getMatrices(self,f):  
        # Slice covarF,covarFB,covarB,meanF,meanB,wF,wB  
        covarF=self.reduceMatrix(self.covar,f,f)  
        meanF=self.reduceMatrix(self.mean,f,[0])  
        b=self.getB(f)  
        covarFB=self.reduceMatrix(self.covar,f,b)  
        wB=self.reduceMatrix(self.w[-1].reshape(-1,1),b,[0])  
        return covarF,covarFB,meanF,wB  
#---------------------------------------------------------------  
    def getB(self,f):  
        return self.diffLists(range(self.mean.shape[0]),f)  
#---------------------------------------------------------------  
    def diffLists(self,list1,list2):  
        return list(set(list1)-set(list2))  
#---------------------------------------------------------------  
    def reduceMatrix(self,matrix,listX,listY):  
        # Reduce a matrix to the provided list of rows and columns  
        if len(listX)==0 or len(listY)==0:return  
        matrix_=matrix[:,listY[0]:listY[0]+1]  
        for i in listY[1:]:  
            a=matrix[:,i:i+1]  
            matrix_=np.append(matrix_,a,1)  
        matrix__=matrix_[listX[0]:listX[0]+1,:]  
        for i in listX[1:]:  
            a=matrix_[i:i+1,:]  
            matrix__=np.append(matrix__,a,0)  
        return matrix__  
#---------------------------------------------------------------  
    def purgeNumErr(self,tol):  
        # Purge violations of inequality constraints (associated with ill-conditioned covar matrix)  
        i=0  
        while True:  
            if i==len(self.w):break  
            w=self.w[i]  
            for j in range(w.shape[0]):  
                if w[j]-self.lB[j]<-tol or w[j]-self.uB[j]>tol:  
                    del self.w[i]  
                    del self.l[i]  
                    del self.g[i]  
                    del self.f[i]  
                    break  
            i+=1  
#---------------------------------------------------------------  
    def purgeExcess(self):  
        # Remove violations of the convex hull  
        i,repeat=0,False  
        while True:  
            if repeat==False:i+=1  
            if i==len(self.w)-1:break  
            w=self.w[i]  
            mu=np.dot(w.T,self.mean)[0,0]  
            j,repeat=i+1,False  
            while True:  
                if j==len(self.w):break  
                w=self.w[j]  
                mu_=np.dot(w.T,self.mean)[0,0]  
                if mu<mu_:  
                    del self.w[i]  
                    del self.l[i]  
                    del self.g[i]  
                    del self.f[i]  
                    repeat=True  
                    break  
                else:  
                    j+=1  
#---------------------------------------------------------------  
    def getMinVar(self):  
        # Get the minimum variance solution  
        var=[]  
        for w in self.w:  
            a=np.dot(np.dot(w.T,self.covar),w)[0,0]  
            var.append(a)  
        return min(var)**.5,self.w[var.index(min(var))]  
#---------------------------------------------------------------  
    def getMaxSR(self):  
        # Get the max Sharpe ratio portfolio  
        #1) Compute the local max SR portfolio between any two neighbor turning points  
        w_sr,sr=[],[]  
        for i in range(len(self.w)-1):  
            w0=np.copy(self.w[i])  
            w1=np.copy(self.w[i+1])  
            kargs={'minimum':False,'args':(w0,w1)}  
            a,b=self.goldenSection(self.evalSR,0,1,**kargs)  
            w_sr.append(a*w0+(1-a)*w1)  
            sr.append(b)  
        return max(sr),w_sr[sr.index(max(sr))]  
#---------------------------------------------------------------  
    def evalSR(self,a,w0,w1):  
        # Evaluate SR of the portfolio within the convex combination  
        w=a*w0+(1-a)*w1  
        b=np.dot(w.T,self.mean)[0,0]  
        c=np.dot(np.dot(w.T,self.covar),w)[0,0]**.5  
        return b/c  
#---------------------------------------------------------------  
    def goldenSection(self,obj,a,b,**kargs):  
        # Golden section method. Maximum if kargs['minimum']==False is passed  
        from math import log,ceil  
        tol,sign,args=1.0e-9,1,None  
        if 'minimum' in kargs and kargs['minimum']==False:sign=-1  
        if 'args' in kargs:args=kargs['args']  
        numIter=int(ceil(-2.078087*log(tol/abs(b-a))))  
        r=0.618033989  
        c=1.0-r  
        # Initialize  
        x1=r*a+c*b;x2=c*a+r*b  
        f1=sign*obj(x1,*args);f2=sign*obj(x2,*args)  
        # Loop  
        for i in range(numIter):  
            if f1>f2:  
                a=x1  
                x1=x2;f1=f2  
                x2=c*a+r*b;f2=sign*obj(x2,*args)  
            else:  
                b=x2  
                x2=x1;f2=f1  
                x1=r*a+c*b;f1=sign*obj(x1,*args)  
        if f1<f2:return x1,sign*f1  
        else:return x2,sign*f2  
#---------------------------------------------------------------  
    def efFrontier(self,points):  
        # Get the efficient frontier  
        mu,sigma,weights=[],[],[]  
        a=np.linspace(0,1,points/len(self.w))[:-1] # remove the 1, to avoid duplications  
        b=range(len(self.w)-1)  
        for i in b:  
            w0,w1=self.w[i],self.w[i+1]  
            if i==b[-1]:a=np.linspace(0,1,points/len(self.w)) # include the 1 in the last iteration  
            for j in a:  
                w=w1*j+(1-j)*w0  
                weights.append(np.copy(w))  
                mu.append(np.dot(w.T,self.mean)[0,0])  
                sigma.append(np.dot(np.dot(w.T,self.covar),w)[0,0]**.5)  
        return mu,sigma,weights 
def opt_port(req_assets, tickers, total_assets):
    temp = dict(zip(cla.w[req_assets].reshape(total_assets), tickers))
    return list(temp.items())[1:]

In [6]:
headers = returns.columns
Lbound = np.zeros(thelen).reshape(thelen,1)
Ubound = np.ones(thelen).reshape(thelen,1)
mean = np.array(returns.mean()).reshape(thelen,1)
covar = np.array(returns.cov())
cla=CLA(mean,covar,Lbound,Ubound)
cla.solve()

In [7]:
opt_port(5,tickers,20)

[(0.03889663048718484, 'AMZN'),
 (0.060282717078837686, 'MA'),
 (0.08958889872939639, 'VZ'),
 (0.10397735174940785, 'PFE'),
 (0.7072544019551733, 'MRK')]

Bailey, D. and López de Prado, M.Bailey, David, and Marcos López de Prado. "An Open-Source Implementation Of The Critical-     Line Algorithm For Portfolio Optimization". Algorithms, vol 6, no. 1, 2013, pp. 169-196. MDPI AG, doi:10.3390/a6010169.         Accessed 07 Aug 2020.

In [ ]:
HRP 2016 